# HR Analytics - Attrition Prediction Model

## Introduction
Attrition is a major concern for organizations due to its implications on recruitment costs, training expenses, and employee morale. This project leverages machine learning models to predict employee attrition using features such as **job satisfaction**, **work-life balance**, and **monthly income**. Insights derived can help organizations implement proactive retention strategies and reduce turnover rates.

### Dataset
The dataset includes various employee attributes and performance indicators fetched from a **PostgreSQL database**. Exploratory Data Analysis (EDA) uncovered patterns and correlations that informed the predictive modeling.

---

## Objective
The primary goals of this project include:
1. **Database Connection**: Efficiently fetch data using **SQLAlchemy** and **psycopg2**.
2. **EDA and Insights**: Uncover the key factors influencing employee attrition.
3. **Model Building**: Train machine learning models:
   - Logistic Regression
   - Logistic Regression with SMOTE
   - Random Forest
4. **Model Evaluation**: Assess performance using metrics like:
   - Accuracy
   - Precision
   - Recall
   - F1-score
   - Area Under the Curve (AUC)
5. **Address Class Imbalance**: Apply **SMOTE** to improve predictions of minority classes (Attrition = True).
6. **Model Saving & Visualization**: Save trained models and visualizations for future reference.

In [1]:
pip install psycopg2 sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

# Parametres
username = "postgres"         
password = "password"         
host = "localhost"            
port = "5432"                 
database = "HR_Analystics"    

# СString connection for SQLAlchemy
connection_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"

# Check connection for SQLAlchemy
print("Testing connection with SQLAlchemy...")
try:
    engine = create_engine(connection_string)
    # Try simple querry.
    test_query = "SELECT 1;"
    with engine.connect() as conn:
        result = conn.execute(test_query)
        print("SQLAlchemy connection successful:", result.fetchall())
except Exception as e:
    print("SQLAlchemy connection failed:", e)

# Check connections with psycopg2
print("\nTesting connection with psycopg2...")
try:
    conn = psycopg2.connect(
        dbname=database,
        user=username,
        password=password,
        host=host,
        port=port
    )
    print("psycopg2 connection successful!")
    conn.close()
except Exception as e:
    print("psycopg2 connection failed:", e)

# Querry if connections successful
query = """
SELECT 
    e.EmpID,
    e.Age,
    e.AgeGroup,
    e.Attrition,
    e.DailyRate,
    e.DistanceFromHome,
    e.Education,
    ef.EducationFieldName AS EducationField,
    e.EmployeeNumber,
    e.EnvironmentSatisfaction,
    e.Gender,
    e.HourlyRate,
    e.JobInvolvement,
    e.JobLevel,
    jr.JobRoleName AS JobRole,
    e.JobSatisfaction,
    ms.MaritalStatusName AS MaritalStatus,
    e.MonthlyIncome,
    e.SalarySlab,
    e.MonthlyRate,
    e.NumCompaniesWorked,
    e.OverTime,
    e.PercentSalaryHike,
    e.PerformanceRating,
    e.RelationshipSatisfaction,
    e.StockOptionLevel,
    e.TotalWorkingYears,
    e.TrainingTimesLastYear,
    e.WorkLifeBalance,
    e.YearsAtCompany,
    e.YearsInCurrentRole,
    e.YearsSinceLastPromotion,
    e.YearsWithCurrManager,
    d.DepartmentName AS Department,
    bt.BusinessTravelName AS BusinessTravel
FROM Employees e
LEFT JOIN Departments d ON e.DepartmentID = d.DepartmentID
LEFT JOIN EducationFields ef ON e.EducationFieldID = ef.EducationFieldID
LEFT JOIN JobRoles jr ON e.JobRoleID = jr.JobRoleID
LEFT JOIN MaritalStatuses ms ON e.MaritalStatusID = ms.MaritalStatusID
LEFT JOIN BusinessTravels bt ON e.BusinessTravelID = bt.BusinessTravelID;
"""

print("\nExecuting query through SQLAlchemy...")
try:
    # Reada data with DataFrame
    df = pd.read_sql_query(query, engine)
    print("Query executed successfully! Here's the data:")
    print(df.head())
except Exception as e:
    print("Query execution failed:", e)

Testing connection with SQLAlchemy...
SQLAlchemy connection failed: Not an executable object: 'SELECT 1;'

Testing connection with psycopg2...
psycopg2 connection successful!

Executing query through SQLAlchemy...
Query executed successfully! Here's the data:
   empid  age agegroup  attrition  dailyrate  distancefromhome  education  \
0  RM297   18    18-25       True        230                 3          3   
1  RM302   18    18-25      False        812                10          3   
2  RM458   18    18-25       True       1306                 5          3   
3  RM728   18    18-25      False        287                 5          2   
4  RM829   18    18-25       True        247                 8          1   

  educationfield  employeenumber  environmentsatisfaction  ...  \
0  Life Sciences             405                        3  ...   
1        Medical             411                        4  ...   
2      Marketing             614                        2  ...   
3  Life Scien

In [3]:
df.head()

empid  age agegroup  attrition  dailyrate  distancefromhome  education  \
0  RM297   18    18-25       True        230                 3          3   
1  RM302   18    18-25      False        812                10          3   
2  RM458   18    18-25       True       1306                 5          3   
3  RM728   18    18-25      False        287                 5          2   
4  RM829   18    18-25       True        247                 8          1   

  educationfield  employeenumber  environmentsatisfaction  ...  \
0  Life Sciences             405                        3  ...   
1        Medical             411                        4  ...   
2      Marketing             614                        2  ...   
3  Life Sciences            1012                        2  ...   
4        Medical            1156                        3  ...   

  stockoptionlevel  totalworkingyears  trainingtimeslastyear  worklifebalance  \
0                0                  0                      2                3   
1                0                  0                      2                3   
2                0                  0                      3                3   
3                0                  0                      2                3   
4                0                  0                      0                3   

  yearsatcompany  yearsincurrentrole yearssincelastpromotion  \
0              0                   0                       0   
1              0                   0                       0   
2              0                   0                       0   
3              0                   0                       0   
4              0                   0                       0   

   yearswithcurrmanager              department     businesstravel  
0                   0.0  Research & Development      Travel_Rarely  
1                   0.0                   Sales      Travel_Rarely  
2                   0.0                   Sales  Travel_Frequently  
3                   0.0  Research & Development         Non-Travel  
4                   0.0  Research & Development         Non-Travel  

[5 rows x 35 columns]

In [4]:
df.describe()

age    dailyrate  distancefromhome    education  \
count  1470.000000  1470.000000       1470.000000  1470.000000   
mean     36.923810   802.485714          9.192517     2.912925   
std       9.135373   403.509100          8.106864     1.024165   
min      18.000000   102.000000          1.000000     1.000000   
25%      30.000000   465.000000          2.000000     2.000000   
50%      36.000000   802.000000          7.000000     3.000000   
75%      43.000000  1157.000000         14.000000     4.000000   
max      60.000000  1499.000000         29.000000     5.000000   

       employeenumber  environmentsatisfaction   hourlyrate  jobinvolvement  \
count     1470.000000              1470.000000  1470.000000     1470.000000   
mean      1024.865306                 2.721769    65.891156        2.729932   
std        602.024335                 1.093082    20.329428        0.711561   
min          1.000000                 1.000000    30.000000        1.000000   
25%        491.250000                 2.000000    48.000000        2.000000   
50%       1020.500000                 3.000000    66.000000        3.000000   
75%       1555.750000                 4.000000    83.750000        3.000000   
max       2068.000000                 4.000000   100.000000        4.000000   

          joblevel  jobsatisfaction  ...  performancerating  \
count  1470.000000      1470.000000  ...        1470.000000   
mean      2.063946         2.728571  ...           3.153741   
std       1.106940         1.102846  ...           0.360824   
min       1.000000         1.000000  ...           3.000000   
25%       1.000000         2.000000  ...           3.000000   
50%       2.000000         3.000000  ...           3.000000   
75%       3.000000         4.000000  ...           3.000000   
max       5.000000         4.000000  ...           4.000000   

       relationshipsatisfaction  stockoptionlevel  totalworkingyears  \
count               1470.000000       1470.000000        1470.000000   
mean                   2.712245          0.793878          11.279592   
std                    1.081209          0.852077           7.780782   
min                    1.000000          0.000000           0.000000   
25%                    2.000000          0.000000           6.000000   
50%                    3.000000          1.000000          10.000000   
75%                    4.000000          1.000000          15.000000   
max                    4.000000          3.000000          40.000000   

       trainingtimeslastyear  worklifebalance  yearsatcompany  \
count            1470.000000      1470.000000     1470.000000   
mean                2.799320         2.761224        7.008163   
std                 1.289271         0.706476        6.126525   
min                 0.000000         1.000000        0.000000   
25%                 2.000000         2.000000        3.000000   
50%                 3.000000         3.000000        5.000000   
75%                 3.000000         3.000000        9.000000   
max                 6.000000         4.000000       40.000000   

       yearsincurrentrole  yearssincelastpromotion  yearswithcurrmanager  
count         1470.000000              1470.000000           1470.000000  
mean             4.229252                 2.187755              4.076871  
std              3.623137                 3.222430              3.498547  
min              0.000000                 0.000000              0.000000  
25%              2.000000                 0.000000              2.000000  
50%              3.000000                 1.000000              3.000000  
75%              7.000000                 3.000000              7.000000  
max             18.000000                15.000000             17.000000  

[8 rows x 24 columns]

### 1. Correlation Analysis

In [5]:
# Select only numeric columns for correlation analysis
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Perform correlation analysis
correlation_matrix = numeric_df.corr()
correlation_matrix

age  dailyrate  distancefromhome  education  \
age                       1.000000   0.010661         -0.001686   0.208034   
dailyrate                 0.010661   1.000000         -0.004985  -0.016806   
distancefromhome         -0.001686  -0.004985          1.000000   0.021042   
education                 0.208034  -0.016806          0.021042   1.000000   
employeenumber           -0.010145  -0.050990          0.032916   0.042070   
environmentsatisfaction   0.010146   0.018355         -0.016075  -0.027128   
hourlyrate                0.024287   0.023381          0.031131   0.016775   
jobinvolvement            0.029820   0.046135          0.008783   0.042438   
joblevel                  0.509604   0.002966          0.005303   0.101589   
jobsatisfaction          -0.004892   0.030571         -0.003669  -0.011296   
monthlyincome             0.497855   0.007707         -0.017014   0.094961   
monthlyrate               0.028051  -0.032182          0.027473  -0.026084   
numcompaniesworked        0.299635   0.038153         -0.029251   0.126317   
percentsalaryhike         0.003634   0.022704          0.040235  -0.011111   
performancerating         0.001904   0.000473          0.027110  -0.024539   
relationshipsatisfaction  0.053535   0.007846          0.006557  -0.009118   
stockoptionlevel          0.037510   0.042143          0.044872   0.018422   
totalworkingyears         0.680381   0.014515          0.004628   0.148280   
trainingtimeslastyear    -0.019621   0.002453         -0.036942  -0.025100   
worklifebalance          -0.021490  -0.037848         -0.026556   0.009819   
yearsatcompany            0.311309  -0.034055          0.009508   0.069114   
yearsincurrentrole        0.212901   0.009932          0.018845   0.060236   
yearssincelastpromotion   0.216513  -0.033229          0.010029   0.054254   
yearswithcurrmanager      0.207127  -0.022798          0.014863   0.065704   

                          employeenumber  environmentsatisfaction  hourlyrate  \
age                            -0.010145                 0.010146    0.024287   
dailyrate                      -0.050990                 0.018355    0.023381   
distancefromhome                0.032916                -0.016075    0.031131   
education                       0.042070                -0.027128    0.016775   
employeenumber                  1.000000                 0.017621    0.035179   
environmentsatisfaction         0.017621                 1.000000   -0.049857   
hourlyrate                      0.035179                -0.049857    1.000000   
jobinvolvement                 -0.006888                -0.008278    0.042861   
joblevel                       -0.018519                 0.001212   -0.027853   
jobsatisfaction                -0.046247                -0.006784   -0.071335   
monthlyincome                  -0.014829                -0.006259   -0.015794   
monthlyrate                     0.012648                 0.037600   -0.015297   
numcompaniesworked             -0.001251                 0.012594    0.022157   
percentsalaryhike              -0.012944                -0.031701   -0.009062   
performancerating              -0.020359                -0.029548   -0.002172   
relationshipsatisfaction       -0.069861                 0.007665    0.001330   
stockoptionlevel                0.062227                 0.003432    0.050263   
totalworkingyears              -0.014365                -0.002693   -0.002334   
trainingtimeslastyear           0.023603                -0.019359   -0.008548   
worklifebalance                 0.010309                 0.027627   -0.004607   
yearsatcompany                 -0.011240                 0.001458   -0.019582   
yearsincurrentrole             -0.008416                 0.018007   -0.024106   
yearssincelastpromotion        -0.009019                 0.016194   -0.026716   
yearswithcurrmanager            0.007760                 0.004529   -0.002151   

                          jobinvolvement  joblevel  jo

### 2. Hypothesis Testing

In [12]:
from scipy.stats import ttest_ind

# Make sure 'attrition' is treated as a categorical column, convert if necessary
df['attrition'] = df['attrition'].map({True: 'Yes', False: 'No'})

# Perform t-test for monthly income by attrition status
stayed = df[df['attrition'] == 'No']['monthlyincome']
left = df[df['attrition'] == 'Yes']['monthlyincome']

# Perform the t-test
t_stat, p_val = ttest_ind(stayed, left, nan_policy='omit')  # Ignore NaN values if any

# Display the test statistic and p-value
print("T-test Statistic:", t_stat)
print("P-value:", p_val)

T-test Statistic: 6.203935765608938
P-value: 7.147363985353811e-10


### 3. Regression Analysis

In [10]:
import statsmodels.api as sm

# Define independent and dependent variables for regression
X = df[['yearsatcompany', 'joblevel']]
y = df['monthlyincome']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          monthlyincome   R-squared:                       0.903
Model:                            OLS   Adj. R-squared:                  0.903
Method:                 Least Squares   F-statistic:                     6838.
Date:                Thu, 21 Nov 2024   Prob (F-statistic):               0.00
Time:                        20:02:38   Log-Likelihood:                -12801.
No. Observations:                1470   AIC:                         2.561e+04
Df Residuals:                    1467   BIC:                         2.562e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -1844.9428     81.213    -22.717      0.000   -2004.250   -1685.636
yearsatcompany     6.5892      7.390      0.892      0.373      -7.907      21.085
joblevel        4022.2457     40.902     98.340      0.000    3942.014    4102.477
==============================================================================
Omnibus:                        1.987   Durbin-Watson:                   1.973
Prob(Omnibus):                  0.370   Jarque-Bera (JB):                2.004
Skew:                          -0.018   Prob(JB):                        0.367
Kurtosis:                       3.177   Cond. No.                         21.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# HR Analytics - Attrition Prediction Model

## Introduction
Attrition is a significant concern for organizations due to its impact on recruitment costs, training expenses, and employee morale. This project utilizes machine learning models to predict employee attrition based on features such as **job satisfaction**, **work-life balance**, and **monthly income**. By identifying key drivers of attrition, organizations can develop strategies to retain top talent and reduce turnover rates.

### Dataset
The dataset contains a variety of employee attributes and performance indicators fetched from a **PostgreSQL database**. Through Exploratory Data Analysis (EDA), patterns and correlations were uncovered, which informed the predictive modeling process.

---

## Objectives
The primary goals of this project are:
1. **Database Connection**: Establish a connection to a PostgreSQL database using **SQLAlchemy** and **psycopg2** to fetch employee data.
2. **EDA and Insights**: Perform detailed exploratory data analysis to uncover key factors influencing employee attrition.
3. **Model Building**: Train machine learning models, including:
   - Logistic Regression
   - Logistic Regression with SMOTE (Synthetic Minority Over-sampling Technique)
   - Random Forest
4. **Model Evaluation**: Evaluate the models using metrics such as:
   - Accuracy
   - Precision
   - Recall
   - F1-score
   - Area Under the Curve (AUC)
5. **Address Class Imbalance**: Use SMOTE to handle the class imbalance in the dataset and improve predictions of "Attrition."
6. **Model Saving & Visualization**: Save trained models and visualizations for future use and decision-making.

---

## Exploratory Data Analysis (EDA)
### Key Insights:
1. **Attrition Count**:
   - **16.1%** of employees have left the company.
   - Indicates a relatively low attrition rate but highlights the need to address factors influencing the 237 employees who left.

2. **Job Satisfaction Distribution**:
   - Most employees report higher job satisfaction levels (scores of 3.0 and 4.0).
   - A notable portion of employees reported low satisfaction (1.0 or 2.0), indicating areas for improvement.

3. **Work-Life Balance**:
   - Most employees rate their work-life balance highly (3.0 or above).
   - A smaller portion reports poor work-life balance (1.0), potentially impacting productivity and retention.

4. **Attrition by Job Role**:
   - Sales Representatives and Laboratory Technicians have the highest attrition rates.
   - Managers and Research Directors show significantly lower attrition rates.

5. **Monthly Income by Attrition**:
   - Employees who left generally have lower monthly incomes compared to those who stayed.
   - Indicates a potential link between compensation and attrition.

6. **Work-Life Balance vs. Job Satisfaction**:
   - Employees with higher work-life balance (3.0 or 4.0) also report higher job satisfaction.
   - Poor work-life balance (1.0) correlates with lower satisfaction scores (1.0 or 2.0).

---

## Models and Performance

### 1. Logistic Regression (Baseline)
- **Accuracy**: 62%
- **Key Metrics**:
  - Precision: **94%** for No Attrition, **27%** for Attrition
  - Recall: **59%** for No Attrition, **79%** for Attrition
  - F1-score: **72%** (No Attrition), **40%** (Attrition)
- **Observations**:
  - Performs well for predicting "No Attrition" but struggles with "Attrition."

---

### 2. Logistic Regression with SMOTE
- **Accuracy**: 65%
- **Key Metrics**:
  - Precision: **91%** (No Attrition), **26%** (Attrition)
  - Recall: **65%** for both classes
  - F1-score: **75%** (No Attrition), **37%** (Attrition)
- **Observations**:
  - SMOTE improved recall for predicting Attrition but still struggles with precision for minority class.

---

### 3. Random Forest
- **Accuracy**: 83%
- **Key Metrics**:
  - Precision: **85%** (No Attrition), **33%** (Attrition)
  - Recall: **97%** (No Attrition), **7%** (Attrition)
  - F1-score: **90%** (No Attrition), **12%** (Attrition)
- **Observations**:
  - Performs well for "No Attrition" predictions but underperforms for "Attrition."

---

## Model Insights and Visualizations
1. **Confusion Matrices**:
   - Logistic Regression: Higher false positives for "Attrition."
   - Random Forest: Low recall for "Attrition" but very high precision for "No Attrition."

2. **Feature Importances**:
   - **Logistic Regression**:
     - Job satisfaction has the largest negative coefficient, indicating it reduces the likelihood of attrition.
   - **Random Forest**:
     - Monthly income is the most important feature, followed by age and total working years.

3. **Performance Curves**:
   - **Precision-Recall Curves**:
     - Logistic Regression with SMOTE improved recall but with some drop in precision.
   - **ROC Curve**:
     - Random Forest achieved an AUC of 0.60, indicating moderate discrimination.

---

## Recommendations
1. **Model Improvements**:
   - Explore advanced models like XGBoost or Neural Networks.
   - Perform hyperparameter tuning to optimize performance.
   - Address class imbalance using techniques like SMOTE or cost-sensitive learning.

2. **Policy Implications**:
   - Focus on improving work-life balance and job satisfaction for employees.
   - Consider targeted salary increases for employees in high-risk roles or lower salary brackets.

3. **Future Work**:
   - Validate models on external datasets to ensure generalizability.
   - Deploy the models for real-time predictions to identify at-risk employees.

---

## Model Saving & Visualizations
- **Models Saved**:
  - Logistic Regression
  - Logistic Regression with SMOTE
  - Random Forest
- **Visualizations**:
  - Confusion matrices, ROC curves, Precision-Recall curves, and feature importance plots have been saved to PDFs for documentation and presentation.
